## **Multiplicação de Matrizes:** Implementação _sequencial_, _Multithreading_ e _Multiprocessos_.

***

Este projeto visa observar as diferenças entre a implementação de algoritmos utilizando as abordagens sequencial, Multithreading e Multiprocessos, realizando testes de implementação e comparativos em termos de desempenho utilizando a linguagem **Python**, para a disciplina **Sistemas Operacionais** do curso de Bacharelado em Tecnologia da Informação, oferecido pela Universidade Federal do Rio Grande do Norte.

**Desenvolvido por:** José Manoel Freitas da Silva

***

### **1. Introdução**

A multiplicação de matrizes é uma operação fundamental em álgebra linear que desempenha um papel crucial em diversos campos da ciência da computação, engenharia e ciências exatas. Uma das principais vantagens da utilização desse método é a sua natureza altamente paralelizável, uma vez que multiplicação de matrizes é bastante suscetível à distribuição de tarefas, seja por meio de threads, em um ambiente multithread, ou por processos. Além disso, a multiplicação de matrizes pode ser escalada para grandes conjuntos de dados e dimensões, o que a torna relevante para problemas de alta complexidade encontrados em pesquisa científica e aplicações industriais.

Desse modo, iremos utilizar de uma de suas propriedades, chamada **independência de dados**, que nos permite calcular cada elemento da matriz resultante de forma independente dos demais dados da matriz, para implementar diversos conceitos da multiprogramação e analisar questões fundamentais de desempenho e otimização algorítmica, ressaltando a sua importância prática e sua capacidade de aproveitar ao máximo os recursos computacionais modernos. 

### **2. Metodologia**

Para esse projeto será feito 3 implementações, sendo elas:

* Uma implementação **Sequencial**, sem a utilização de multiprocessos ou multithreading;
* Uma implementação utilizando **Threads**;
* Uma implementação utilizando **Processos**.

Cada uma das implementações receberá duas matrizes **M1** e **M2** de tamanho **C**x**L** variados, com valores e dimensões randomizadas por meio de um algorítimo auxiliar implementado em **Python**. Para as implementações em multiprocessamento será utilizado ainda uma segunda variável **P**, que determina a quantidade de elementos que serão processados em cada thread ou processo criado, de modo a evitar problemas oriundos do excesso de processos abertos, frequentemente vistos em matrizes consideravelmente grandes.

Para quantificar o desempenho de cada implementação será medido o tempo de execução em cada uma das metodologias, realizando a plotagem de gráficos de desempenho com o intuito de visualizar o **tempo médio geral** e o **tempo médio em função de P**.

### **3. Considerações iniciais**

Nesse projeto, devido ao seu caráter analítico, será utilizado a linguagem de programação **Python**, aliada ao **Jupyter Notebook**, onde iremos observar o comportamento de diferentes formas de implementação de um mesmo algoritmo com o intuito de analisar as vantagens e desvantagens de **aplicações sequencias** e **paralelas**.

A escolha dessas duas ferramentas se deve ao fato de permitir produzir gráficos e relatórios com facilidade quando utilizadas em conjunto com algumas bibliotecas, como a _multiprocessing_, _threading_,_matplotlib_ e a _numpy_, que terá um papel fundamental na criação de objetos e elementos utilizados durante os testes.

### **4. Multiplicação de matrizes**

#### **4.1 Operações entre matrizes**

Uma matriz pode ser definida como uma estrutura matemática que organiza elementos em **linhas** e **colunas**, formando uma grade bidimensional. Nela cada elemento é identificado por sua posição única na matriz através do índice de linhas e colunas. Por exemplo, na matriz $A$, de dimensões 3x3, o elemento $A[i][j]$ é identificado pela linha $i$ e coluna $j$.

\begin{equation*}

A = \begin{bmatrix}
1 & 2 & 3 \\
4 & 5 & 6 \\
7 & 8 & 9

\end{bmatrix}

\end{equation*}

Assim, o elemento $A_{1,2}$ da matriz $A$ é igual a **2**.

Essa estrutura também está suscetível a algumas operações, como a multiplicação entre matrizes, que será utilizada como objeto de estudo.

A **multiplicação de matrizes** é uma operação que corresponde ao  produto de duas matrizes $A$ e $B$, para produzir uma terceira matriz $C$. Para calcular o elemento $C_{i,j}$ da matriz resultante $C$, devemos multiplicar os elementos da linha $i$ da matriz $A$ pelos elementos da coluna $j$ da matriz $B$, somando os produtos. Matematicamente, isso pode ser expresso como:

\begin{equation*}
C_{i,j} = A_{i,1} \cdot B_{1,j} + A_{i,2} \cdot B_{2,j} + \ldots + A_{i,n} \cdot B_{n,j}
\end{equation*}

Esse processo é repetido para todos os elementos da matriz resultante $C$, e o resultado é uma nova matriz que é o produto das matrizes $A$ e $B$, com a quantidade colunas de $A$ e linhas de $B$, assim como veremos nos algoritmos implementados a seguir.

#### **4.2 Algoritmos e implementações** 

##### **4.2.1 Importando recursos**

Para a implementação dos algoritmos a seguir utilizaremos as seguintes bibliotecas:

* **NumPy:** Biblioteca em Python que fornece suporte para arrays multidimensionais e funções matemáticas de alto desempenho para manipulação de dados numéricos;

In [104]:
import numpy as np

##### **4.2.2 Criando matrizes**

Nesse estudo serão utilizadas matrizes de inteiros construídas de forma randômica através da função **_get_random_matrix_**, que utiliza a função **_random_**, da biblioteca **NumPy**. Para fins de estudos todos os elementos irão possuir um valor fixado entre 1 e 10. Além das matrizes randômicas, também iremos utilizar matrizes inicializadas com zeros por meio da função **_initialize_matrix_**, necessária para garantir que todos os elementos da matriz resultante tenham um valor inicial conhecido e consistente.

Vale ressaltar que para ocorrer a multiplicação entre matrizes a quantidade de linhas de $A$ e colunas de $B$ deve possuir o mesmo valor, seguindo a propriedade a seguir:

\begin{equation*}
[A_{m \times n}] \cdot [B_{n \times p}] = [C_{m \times p}]
\end{equation*}

Desse modo, para satisfazer essa condição, iremos utilizar a variável **_cond_** para garantir que ambas as matrizes criadas obedeçam essa propriedade.

In [105]:
def get_random_matrix(rows, cols):
    # Cria matrizes aleatórias com valores 1 e 10
    return np.random.randint(1, 11, size = (rows, cols))

In [106]:
def initialize_matrix(rows, cols):
    return np.zeros((rows, cols))

In [107]:
# Limite para o tamanho das matrizes
limit = 5

# Condição para existencia do produto
cond = np.random.randint(2, limit)

# Cria as matrizes aleatórias
matrix1 = get_random_matrix(np.random.randint(2, limit), cond)
matrix2 = get_random_matrix(cond, np.random.randint(2, limit))

# Imprima as matrizes
print("Matriz 1 de tamanho", matrix1.shape)
print(matrix1)
print("\n")
print("Matriz 2 de tamanho", matrix2.shape)
print(matrix2)


Matriz 1 de tamanho (3, 2)
[[8 8]
 [4 4]
 [4 4]]


Matriz 2 de tamanho (2, 4)
[[6 8 2 7]
 [5 9 7 9]]
